# Adjoin

```
from firedrake import *
from firedrake.adjoint import *
from firedrake.petsc import PETSc
import numpy as np

continue_annotation()

def get_face_size(mesh):
    """
    Get the sizes of each faces of mesh.
    """
    dim = mesh.geometric_dimension()
    fe = FacetArea(mesh)

    U = FunctionSpace(mesh, "Discontinuous Lagrange Trace", 0)
    # TODO: Too slow!
    he = Function(U)
    pause_annotation()
    he.project(fe) # Diameter of the element
    if dim == 3:
        he.dat.data[:] = np.sqrt(he.dat.data[:])
    continue_annotation()
    return he


def get_Penalty_term(mesh, uh, vh, gamma):
    degree = len(gamma)
    n = FacetNormal(mesh)
    he = get_face_size(mesh) # May also be Constant(1)
    # he = Constant(1)
    gpu = grad(uh)
    gpv = grad(vh)
    jgun = dot(gpu('+'), n('+')) + dot(gpu('-'), n('-')) # jump(gpu,n)
    jgvn = dot(gpv('+'), n('+')) + dot(gpv('-'), n('-')) # jump(gpv,n)
    J = avg(gamma[0] * he) * inner(jgun, jgvn) * dS # (domain=mesh)
    for i in range(1, degree):
        # gpu = dot(n, grad(gpu))
        # gpv = dot(n, grad(gpv))
        # jgun = dot(gpu('+'), n('+')) + ((-1)**i) * dot(gpu('-'), n('-'))
        # jgvn = dot(gpv('+'), n('+')) + ((-1)**i) * dot(gpv('-'), n('-'))
        gpu = grad(gpu)
        gpv = grad(gpv)
        gpu_ = dot(n, gpu)
        gpv_ = dot(n, gpv)
        for _ in range(1, i):
            gpu_ = dot(n, gpu_)
            gpv_ = dot(n, gpv_)
        jgun = dot(gpu_('+'), n('+')) + ((-1)**i) * dot(gpu_('-'), n('-'))
        jgvn = dot(gpv_('+'), n('+')) + ((-1)**i) * dot(gpv_('-'), n('-'))
        J = J + avg(gamma[i] * he**(2*i+1)) * inner(jgun, jgvn) * dS # (domain=mesh)
    
    return J


# Define a simple mesh
N = 16
mesh = UnitCubeMesh(N, N, N)

degree = 1
k = 10
# Plane waves
phi = np.linspace(0, 11 * pi / 6, 6)
theta = np.linspace(0, 5 * pi / 6, 6)

dim = mesh.geometric_dimension()

x = SpatialCoordinate(mesh)
V = FunctionSpace(mesh, "CG", degree)
G = VectorFunctionSpace(mesh, "Discontinuous Lagrange Trace", degree=0, dim=degree)

W = V*V

u_r, u_i = TrialFunctions(W)
v_r, v_i = TestFunctions(W)

gamma = Function(G, name='gamma').assign(0.07)
lower_gamma = Function(G, name='lower_gamma').assign(-1.0)
upper_gamma = Function(G, name='uppler_gamma').assign(1.0)
control = Control(gamma)

# Assemble
k_c = Constant(k)

n = FacetNormal(mesh)

th = Constant(0.0)
ph = Constant(0.0)
if dim == 3:
    u_pw_r = cos(k_c * (x[0] * cos(ph) * sin(th) + x[1] * sin(ph) * sin(th) + x[2] * cos(th)))
    u_pw_i = sin(k_c * (x[0] * cos(ph) * sin(th) + x[1] * sin(ph) * sin(th) + x[2] * cos(th)))
elif dim == 2:
    u_pw_r = cos(exp(k_c * (x[0] * cos(ph) + x[1] * sin(ph))))
    u_pw_i = sin(exp(k_c * (x[0] * cos(ph) + x[1] * sin(ph))))

g_r = dot(grad(u_pw_r), n) + k_c*u_pw_i
g_i = dot(grad(u_pw_i), n) - k_c*u_pw_r

def helmholtz(u, v):
    return inner(grad(u), grad(v))*dx - k_c * k_c * inner(u, v)*dx

A = helmholtz(u_r, v_r) + helmholtz(u_i, v_i) \
    + k_c*inner(u_i, v_r)*ds - k_c*inner(u_r, v_i)*ds \
    - inner(g_r, v_r)*ds + inner(g_i, v_i)*ds \
    + get_Penalty_term(mesh, u_r, v_r, gamma) \
    + get_Penalty_term(mesh, u_i, v_i, gamma)

u_h = Function(W, name='u_h')

prob = LinearVariationalProblem(lhs(A), rhs(A), u_h)
solver = LinearVariationalSolver(prob, options_prefix='gamma')

u_h_r, u_h_i = u_h[0], u_h[1]

J_expr = grad(u_pw_r - u_h_r)**2*dx + (u_pw_r - u_h_r)**2*dx \
        + grad(u_pw_i - u_h_i)**2*dx + (u_pw_i - u_h_i)**2*dx

J = 0
for _th in theta:
    for _ph in phi:
        th.assign(_th)
        ph.assign(_ph)
        solver.solve()

        Jtemp = assemble(J_expr)
        J += Jtemp

# tape = get_working_tape()
# for i, block in enumerate(tape._blocks):
#     print("Block {:2d}: {:}".format(i, type(block)))

Jhat = ReducedFunctional(J, control)
get_working_tape().progress_bar = ProgressBar

# https://github.com/dolfin-adjoint/pyadjoint/blob/b9574fbd2c36f47da66e226854e997decf32bbf8/pyadjoint/optimization/optimization.py#L8
g_opt = minimize(Jhat,
                 bounds=[[lower_gamma], [upper_gamma]],
                 options={"maxiter": 100},
                 derivative_options={"riesz_representation": 'l2'})

with g_opt.dat.vec_ro as vec:
    PETSc.Sys.Print(vec.min(), vec.max())
```